# 1. Getting Started

## 1.1 Loading a Model and Inspecting It

To begin with, __masspy__ incorporates <font color=#1E90FF>metabolites, reactions, initial conditions </font>for a given metabolite and rate equations for each reaction in order to create models for simulations of cell reactions. To load a test model, type: 

In [9]:
from mass import MassReaction, MassMetabolite, MassModel
from mass.test import create_test_model
#Incorporates RBC Core Model as a test model
model= create_test_model("textbook")

The <font color=#1E90FF>reactions, metabolites and initial contditions</font> reactions, metabolites and initial contditions are attributes of the masspy model which corresponds to a special type of list called a <font color='#556B2F'>cobra.DictList</font>, and each one is made up of <font color=#4682B4>mass.reaction</font> and <font color='#4682B4'>mass.metabolite</font> and many more respectively.


In [2]:
print(len(model.reactions))
print(len(model.metabolites))

53
48


When using Jupyter Notebook, this type of information is rendered in a table. 

In [3]:
model

Name,Core_RBC_Model
Memory address,0x012c362cc0
Stoichiometric Matrix,48x53
Matrix Rank,44
Matrix Type,"dense, float64"
Number of Metabolites,48
Number of Reactions,53
Number of Initial Conditions,48
Number of Forward Rate Constants,53
Number of Equilibrium Constants,53
Number of Irreversible Reactions,11


Just like a regular list, objects in the DictList can be retrieved by indexing. For example, to get the 30th reaction in the model (at index 29 because of *0-indexing*)

In [4]:
model.reactions[29]

Reaction identifier,DPGM
Name,Diphosphoglyceromutase
Memory address,0x012c3999b0
Subsystem,Hemoglobin
Stoichiometry,"_13dpg_c <=> _23dpg_c + h_c 3-Phospho-D-glyceroyl phosphate <=> 2,3-Disphospho-D-glycerate + H+"
GPR,
Kinetic Reversibility,True


Additionally, items can be retrieved by their ID using the <font color=#B22222>DictList.get_by_id()</font> function. For example, to get the cytosolic atp metabolite object in our RBC core model(the ID is "atp_c"), we can do the following:

In [5]:
model.metabolites.get_by_id("atp_c")

MassMetabolite identifier,atp_c
Name,ATP
Memory address,0x012c36a588
Formula,C10H12N5O13P3
Compartment,c
Initial Condition,1.6
Gibbs formation energy,None
In 8 reaction(s),"ADNK1, PGK, PFK, PYK, ADK1, PRPPS, ATPM, HEX1"


Users with an interactive shell such as Jupyter Notebook will be able to tab complete list elements inside a list. While this is not recommended behavior for most code because of the possibility of characters like "-" inside ids, this is very useful while in an interactive prompt:

In [6]:
print(model.reactions.EX_lac__L_e.forward_rate_constant)
print(model.reactions.EX_lac__L_e.equilibrium_constant)
#put in forward and reverse rate constants

5.789814814814814
1


__MassPy__ functionality allows us to call up it's specific attributes: a portion of class. Class and object functions can be visualized as compartmentalization and organizational methods. Classes program functions within the objects to comprise an attribute; similar to organizing a compartmentalized network of systems. Here, we display a function which only calls the attributes associated with __masspy__,

In [7]:
def attribute(obj):
    
    "A portion of the class; classes program functions within the objects to comprise an attribute"
    pass

To see all the available attributes within the model being out object, we can do:  


In [ ]:
dir(MassReaction)

To inspect the MassModel objects, various methods exist as outline above; to highlight a few, 

In [ ]:
model.reactions

As part of the DictList function, we can view all of the reactions if we so choose to; this would require us to not index the particular reaction and list out all reactions if necessary. 

In [ ]:
model.metabolites

To examine the stoichiometric matrix, we can do,

In [ ]:
model.S

This would be better viewed through the <font color='#8A2BE2'>pandas.DataFrame</font> table; to do so, 

In [ ]:
import pandas as pd

In [ ]:
model.update_S(matrix_type="DataFrame", update_model=True)

We can't really observe all of the reactions taking place here; therefore, we can set the amount of rows and columns displayed so that we can observe all of the values of said reaction,

In [ ]:
pd.set_option("display.max_rows",100)
pd.set_option("display.max_columns",100)
pd.set_option("display.max_colwidth",-1)
pd.options.display.float_format = '{:,.3f}'.format #sets the amount of decimal places numbers have

When we call for the data frame again, we see that all the reactions and the corresponding concentrations can be viewed in the table now,

In [ ]:
model.update_S(matrix_type="DataFrame", update_model=True)

Model rates can be examined by iterating using a "for" loop,

In [ ]:
for rxn, rate in model.rates.items():
    print("%s: %s" % (rxn.id, rate))

The above loop displays  rates and the datatype is a *dictionary list*. Dictionary (from now on referred to as *Dict*) list refers to the dictionary input which you will use heavily in __masspy__. It's important to note the distinction in the type of data that you call upon because this can make extraction of information much easier.

In [ ]:
model.rates[model.reactions.PPM]

Using <font color=#FF00FF>dict.get(key,None)</font> methodology ensures more efficient iterations and it returns the value or any given key. If key is not present, it would return "None". In our case, we have reactions as keys and rates acting as a standard dictionary.

In [ ]:
print(model.rates.get(model.reactions.PGL,None))
print(model.rates.get("model.reactions.PCC",None))

It's essential to note that the datatype for rates and ODE's are based on preceding rate law formulations and thereby are sympy symbolic objects. Symbolic objects are inherited from a basic object such as lists, dictionaries, etc. Comprising all these basic object in sympy results in the sympy expression we see below.   

In [ ]:
import sympy as sym
from sympy import init_printing

In [ ]:
print(model.rates[model.reactions.HEX1])
print(type(model.rates[model.reactions.HEX1]))
print(type(str(model.rates[model.reactions.HEX1])))
print(isinstance(model.rates[model.reactions.HEX1], str))

In [ ]:
print(type(model.odes[model.metabolites.glc__D_c]))
print(type(str(model.odes[model.metabolites.glc__D_c])))
print(isinstance(model.odes[model.metabolites.glc__D_c], str))

Pretty printing can be utilized in these functions in order to ensure that the 

In [ ]:
from sympy import pprint
eqn = model.rates[model.reactions.HEX1]
sym.pprint(eqn)

The same goes for ODE's; the following function is not being printed for the sake of organization

In [ ]:
for metabolite, ode in model.odes.items():
    print("%s:%s" % (metabolite.id,ode))

Similarly, one ODE can be pulled out,

In [ ]:
model.odes[model.metabolites.glc__D_c] #*metabolites don't have ODE until they are put in*

## 1.2 Reactions

We will consider the reaction glucose-6-phosphate isomerase, which interconverts glucose-6-phosphate to fructose-6-phosphate. The reaction model for this reaction in our test model is PGI

In [ ]:
pgi= model.reactions.get_by_id("PGI")
pgi

We can view the full name and reaction catalyzed as strings.

In [ ]:
print(pgi.name)
print(pgi.reaction)

We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [ ]:
pgi.check_mass_balance()

In order to add a metabolite, we parse through a *dict list* with the metabolite object and its coefficient.

In [ ]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"):-1})
pgi.reaction

The reaction is no longer mass balanced.

In [ ]:
pgi.check_mass_balance()

We can remove metabolite and the reaction can be balanced once again.

In [ ]:
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"):-1})
print(pgi.reaction)
print(pgi.check_mass_balance())

Further inspection of the reaction itself can be accomplished through several methods listed below:

In [ ]:
pgi.id

In [ ]:
pgi.reactants

In [ ]:
pgi.products

In [ ]:
pgi.stoichiometry

As stated above, <font color=#800080>dir(MassReaction)</font> can pull up the documentation for MassReaction Class.

In [10]:
dir(model.reactions)

['ADA',
 'ADK1',
 'ADNK1',
 'ADPT',
 'AMPDA',
 'ATPM',
 'DM_nadh',
 'DPGM',
 'DPGase',
 'ENO',
 'EX_ade_e',
 'EX_adn_e',
 'EX_co2_e',
 'EX_h2o_e',
 'EX_h_e',
 'EX_hxan_e',
 'EX_ins_e',
 'EX_lac__L_e',
 'EX_nh3_e',
 'EX_o2_e',
 'EX_pi_e',
 'EX_pyr_e',
 'FBA',
 'G6PDH2r',
 'GAPD',
 'GND',
 'GSHR',
 'GTHOr',
 'HBDPG',
 'HBO1',
 'HBO2',
 'HBO3',
 'HBO4',
 'HEX1',
 'LDH_L',
 'NTD11',
 'NTD7',
 'PFK',
 'PGI',
 'PGK',
 'PGL',
 'PGM',
 'PPM',
 'PRPPS',
 'PUNP5',
 'PYK',
 'RPE',
 'RPI',
 'S_glc__D_e',
 'TALA',
 'TKT1',
 'TKT2',
 'TPI',
 '__add__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__re

## 1.3 Metabolites 

We will consider cytosilic ATP as the metabolite and use id "atp_c" within the following test model. 

In [ ]:
atp= model.metabolites.get_by_id("atp_c")
atp

We can print out the metabolite name and compartment (cytosol in this case) directly as string. 

In [ ]:
print(atp.name)
print(atp.compartment)

We can also determine the charge of ATP within the model

In [ ]:
atp.charge

Chemical formula of the metabolite can be observed as well. 

In [ ]:
print(atp.formula)

The reactions attribute gives a frozenset of all reactions using given metabolite. We can use this to count the number of reactions which use ATP.

In [ ]:
len(atp.reactions)

A metabolite like G6P will participate in fewer reactions.

In [ ]:
model.metabolites.get_by_id("g6p_c").reactions

## 1.4 Parameters & Initial Conditions

Through using __MassPy__ specific attributes, one can inspect initial conditions and parameters placed within the model.

In [ ]:
model.initial_conditions

From the list above, we can inspect one specific initial condition instead of all of them: 

In [ ]:
model.initial_conditions[model.metabolites._13dpg_c]
#can also get initial conditions through met.ic
#important to point out diff b/w met.ic and model.ic
#Include kf/ KEq

These tools are espiecially important in determining how to inspect models; in the next workbook, we will discuss how to create models from the ground up and contextualize the methods to derive the model. 

Going through this initialized notebook, we see that a few important distinctions should be noted when comparing other python programs to __MassPy__. Functionality and inspection of models differ from __CobraPy__ to __MassPy__, and the nuances in the syntax is important to observe. We discussed how to load a model and pull up the models along with it's different attributes. After inspecting reactions, metabolites and pandas dataframes, we talked about datatypes and how to index with different methods. thereafter, we went over couple of functionalities for reactions and metabolites with respect to the attributes they have such as checking mass balance and finally ended with talking a little about initial conditions. 